In [1]:
import torch
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:
root_dir = "../forest_fire/Training and Validation"

# image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resizing for ResNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ResNet normalization
])

dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# splits training data into 3:1 ratio
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# dataloaders for training
# batch_size = 32
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)

# for checking purposes
print(f"Total images: {len(dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")


/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total images: 1832
Training images: 1374
Validation images: 458


In [3]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# sets device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# moves the model to device
model = model.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

# training/validation
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train


    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)


            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / total_val
    val_accuracy = correct_val / total_val

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


Validating: 100%|██████████| 29/29 [00:02<00:00, 10.52it/s]


Epoch [1/20], Train Loss: 0.2393, Train Accuracy: 0.9178, Val Loss: 0.0837, Val Accuracy: 0.9738


Validating: 100%|██████████| 29/29 [00:01<00:00, 22.96it/s]


Epoch [2/20], Train Loss: 0.1237, Train Accuracy: 0.9585, Val Loss: 0.2056, Val Accuracy: 0.9541


Validating: 100%|██████████| 29/29 [00:01<00:00, 23.78it/s]


...

Epoch [18/20], Train Loss: 0.0216, Train Accuracy: 0.9956, Val Loss: 0.0366, Val Accuracy: 0.9913


Validating: 100%|██████████| 29/29 [00:01<00:00, 24.66it/s]


Epoch [19/20], Train Loss: 0.0279, Train Accuracy: 0.9891, Val Loss: 0.0412, Val Accuracy: 0.9891


Validating: 100%|██████████| 29/29 [00:01<00:00, 24.47it/s]

Epoch [20/20], Train Loss: 0.0152, Train Accuracy: 0.9956, Val Loss: 0.0453, Val Accuracy: 0.9891


In [4]:
# for saving the model
model_save_path = "wildfire_resnet50.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to wildfire_resnet50.pth


In [ ]:
# for loading the model
model_load_path = "wildfire_resnet50.pth"
model.load_state_dict(torch.load(model_load_path))
# model = model.to(device)